# Week 3 — Agents & Tools

**Course:** LangChain for AI Applications  
**Week Focus:** Build autonomous agents that can use tools, make decisions, and solve complex multi-step problems.

---

## 🎯 Learning Objectives

By the end of this week, you will:
- Understand the ReAct (Reasoning + Acting) framework
- Build custom tools for calculations, searches, and queries
- Create agent executors that autonomously choose tools
- Handle errors and edge cases gracefully
- Implement retry logic and fallback strategies
- Build a research assistant that answers complex questions

## 📊 Real-World Context

**The Challenge:** Your data analytics team receives 200+ ad-hoc requests daily:
- 45% require database queries ("What were Q3 sales?")
- 30% need calculations ("Growth rate vs last year?")
- 15% require web research ("Latest industry trends?")
- 10% combine multiple sources (SQL + calculations + reports)

**Problems with static chains:**
- ❌ Fixed workflows can't adapt to different questions
- ❌ Can't decide which tools to use
- ❌ Fail on unexpected inputs
- ❌ Require manual intervention

**The Solution:** An autonomous research agent that:
1. **Analyzes** the question and determines required tools
2. **Executes** tools in the right sequence
3. **Combines** results from multiple sources
4. **Handles** errors and tries alternative approaches
5. **Delivers** comprehensive, accurate answers

**Business Impact:**
- ⏱️ Reduce avg response time from 2 hours → 30 seconds
- 💰 Save $180K/year in analyst time
- 📊 Answer 10x more queries with same team
- 🎯 95% accuracy on routine questions

Companies like **GitHub Copilot, Perplexity, and Replit** use autonomous agents to assist millions of users.

In [ ]:
from IPython.display import HTML
HTML('''
<style>
.jp-RenderedHTMLCommon h2 {
    color: #2c3e50;
    border-bottom: 2px solid #3498db;
    padding-bottom: 10px;
    margin-top: 30px;
}
.jp-RenderedHTMLCommon h3 {
    color: #34495e;
    margin-top: 20px;
}
.jp-RenderedHTMLCommon code {
    background-color: #f8f9fa;
    padding: 2px 6px;
    border-radius: 3px;
    font-family: 'Courier New', monospace;
}
.jp-RenderedHTMLCommon pre {
    background-color: #f8f9fa;
    border-left: 4px solid #3498db;
    padding: 15px;
    border-radius: 5px;
}
.exercise-box {
    background-color: #fff3cd;
    border-left: 5px solid #ffc107;
    padding: 15px;
    margin: 20px 0;
    border-radius: 5px;
}
.scenario-box {
    background-color: #d1ecf1;
    border-left: 5px solid #17a2b8;
    padding: 15px;
    margin: 20px 0;
    border-radius: 5px;
}
.agent-box {
    background-color: #e8f5e9;
    border-left: 5px solid #4caf50;
    padding: 15px;
    margin: 20px 0;
    border-radius: 5px;
}
.tool-box {
    background-color: #f3e5f5;
    border-left: 5px solid #9c27b0;
    padding: 15px;
    margin: 20px 0;
    border-radius: 5px;
}
</style>
''')

## 🔍 Part 1: Understanding the ReAct Framework

### The ReAct Pattern (Reasoning + Acting)

ReAct is an iterative loop where the agent:
1. **Thinks** (Reasoning): Analyzes the problem and decides which tool to use
2. **Acts**: Calls the tool
3. **Observes**: Sees the result
4. **Repeats**: Uses the observation to decide next steps

This cycle continues until the agent reaches a final answer.

**Example ReAct Loop:**

```
User: "What's the product launch date of the top AI startup funded in 2024?"

Agent Thinks: "I need to:
  1. Find the top AI startup funded in 2024
  2. Get their product launch date
  Let me search for this information."

Agent Acts: Uses search_web tool
  Query: "top AI startup funded 2024"

Agent Observes: Gets results about Anthropic Series D, OpenAI funding, etc.

Agent Thinks: "OpenAI's Series D was the largest AI funding round in 2024 ($6.6B).
  Now I need their product launch date. But they're already established.
  Let me search for 2024 new AI startups instead."

Agent Acts: Uses search_web again
  Query: "new AI startup 2024 highest funding"

Agent Observes: Gets results about Mistral AI, Anthropic updates, etc.

Agent Thinks: "I found good information. Mistral AI raised $415M in 2024.
  They launched their API service in June 2024.
  This appears to be the most recent top AI startup launch."

Agent Final Answer: "Mistral AI, funded $415M in 2024, launched their API service in June 2024."
```

### Why Agents Are More Powerful Than Chains

| Aspect | Chains | Agents |
|--------|--------|--------|
| **Workflow** | Fixed, predetermined | Dynamic, adaptive |
| **Tool Selection** | Predefined order | LLM decides based on input |
| **Error Handling** | Fails on unexpected input | Can reason and retry |
| **Complexity** | Simple tasks (2-3 steps) | Complex multi-step reasoning |
| **Decision Making** | Manual routing | Automated decision logic |
| **Flexibility** | Low - need new chain for variations | High - same agent handles variations |

**Real-world example:**
- **Chain approach:** "If customer asks about returns, run returns_chain. If about shipping, run shipping_chain."
- **Agent approach:** "Agent analyzes question and decides - does it need product lookup? Customer history? Shipping data? Acts accordingly."

## 📚 Part 2: Building Tools

<div class="tool-box">
<strong>🛠️ What Are Tools?</strong><br><br>
Tools are functions that agents can call to:
<ul>
<li><strong>Query databases:</strong> "SELECT * FROM sales WHERE quarter='Q3'"</li>
<li><strong>Perform calculations:</strong> Compute growth rates, projections</li>
<li><strong>Search information:</strong> Web search, document lookup</li>
<li><strong>Interact with APIs:</strong> Get weather, stock prices, user data</li>
<li><strong>Retrieve data:</strong> From files, databases, knowledge bases</li>
</ul>
</div>

In [ ]:
# Building Custom Tools

from langchain_core.tools import tool
from typing import Optional
import pandas as pd
import numpy as np

# Simulated sales database
SALES_DATA = pd.DataFrame({
    'region': ['North', 'South', 'East', 'West', 'North', 'South', 'East', 'West'],
    'quarter': ['Q3', 'Q3', 'Q3', 'Q3', 'Q4', 'Q4', 'Q4', 'Q4'],
    'revenue': [250000, 180000, 320000, 200000, 280000, 195000, 350000, 220000],
    'deals': [15, 12, 20, 14, 17, 13, 22, 16]
})

print("📊 Sample Sales Database:")
print(SALES_DATA)
print()

# 🔧 Tool 1: Query Sales Data
@tool
def query_sales_data(region: Optional[str] = None, quarter: Optional[str] = None) -> str:
    """Query sales data by region and/or quarter.
    
    Args:
        region: Region name (North, South, East, West) or None for all
        quarter: Quarter (Q3, Q4, etc.) or None for all
    
    Returns:
        Formatted sales data matching criteria
    """
    data = SALES_DATA.copy()
    
    if region:
        data = data[data['region'] == region]
    if quarter:
        data = data[data['quarter'] == quarter]
    
    if data.empty:
        return f"No data found for region={region}, quarter={quarter}"
    
    return data.to_string()

# 🔧 Tool 2: Calculate Growth Rate
@tool
def calculate_growth_rate(current_value: float, previous_value: float) -> str:
    """Calculate growth rate percentage.
    
    Args:
        current_value: Current period value
        previous_value: Previous period value
    
    Returns:
        Growth rate as percentage
    """
    if previous_value == 0:
        return "Cannot calculate growth from zero baseline"
    
    growth = ((current_value - previous_value) / previous_value) * 100
    return f"Growth rate: {growth:.2f}%"

# 🔧 Tool 3: Aggregate Region Performance
@tool
def aggregate_regional_performance(quarter: str) -> str:
    """Get performance summary for all regions in a quarter.
    
    Args:
        quarter: Quarter to analyze (Q3, Q4, etc.)
    
    Returns:
        Regional performance rankings
    """
    data = SALES_DATA[SALES_DATA['quarter'] == quarter]
    if data.empty:
        return f"No data for quarter {quarter}"
    
    summary = data.sort_values('revenue', ascending=False)
    result = f"\n{quarter} Regional Performance:\n"
    result += "="*50 + "\n"
    for idx, row in summary.iterrows():
        result += f"{row['region']:10} | Revenue: ${row['revenue']:>10,.0f} | Deals: {row['deals']:>3}\n"
    
    total_revenue = data['revenue'].sum()
    total_deals = data['deals'].sum()
    result += "="*50 + "\n"
    result += f"{'TOTAL':10} | Revenue: ${total_revenue:>10,.0f} | Deals: {total_deals:>3}\n"
    
    return result

print("✅ Tool 1: Query Sales Data")
print(query_sales_data(region="North"))
print()

print("✅ Tool 2: Calculate Growth")
print(calculate_growth_rate(current_value=280000, previous_value=250000))
print()

print("✅ Tool 3: Aggregate Regional Performance")
print(aggregate_regional_performance("Q4"))

## 🛠️ Part 3: Creating the Research Agent

### Step 1: Define Tools for the Agent

In [ ]:
# Create a list of tools the agent can use
tools = [
    query_sales_data,
    calculate_growth_rate,
    aggregate_regional_performance
]

print("📋 Available Tools:")
for tool_func in tools:
    print(f"\n🔧 {tool_func.name}")
    print(f"   Description: {tool_func.description}")
    if hasattr(tool_func, 'args_schema'):
        print(f"   Arguments: {tool_func.args_schema}")

### Step 2: Create the Agent

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_react_agent, AgentExecutor
from langchain.llms.fake import FakeListLLM

# Create agent prompt
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are an expert sales analyst assistant for a SaaS company.
    
    Your capabilities:
    - Query sales data by region and quarter
    - Calculate growth rates and trends
    - Provide regional performance analysis
    
    When answering questions:
    1. Use the tools to gather data
    2. Analyze the results
    3. Provide clear, actionable insights
    4. Always cite specific numbers
    
    Answer with confidence but admit when you don't have data.
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create fake LLM for demo (in production, use OpenAI, Claude, etc.)
llm = FakeListLLM(responses=[
    "I'll analyze the Q4 performance data for you.",
    "Let me calculate the growth rate from Q3 to Q4.",
    "Based on the analysis, the East region showed the strongest performance with $350,000 in Q4 revenue."
])

# Create the agent
agent = create_react_agent(llm, tools, agent_prompt)

# Create agent executor (handles tool calls and loops)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("✅ Research Agent Created!")
print(f"   Agent type: ReAct (Reasoning + Acting)")
print(f"   Tools available: {len(tools)}")
print(f"   Max iterations: 5")

### Step 3: Test the Agent with Real Queries

In [ ]:
# Test Query 1: Simple data lookup
print("🧪 QUERY 1: Simple Data Lookup")
print("="*70)
print("User: What were the Q3 sales for the North region?")
print()

response1 = agent_executor.invoke({
    "input": "What were the Q3 sales for the North region?",
    "chat_history": [],
    "agent_scratchpad": ""
})

print(f"\nAgent Response: {response1['output']}")
print()
print()

# Test Query 2: Multi-step analysis
print("🧪 QUERY 2: Multi-Step Analysis")
print("="*70)
print("User: Which region had the best Q4 performance and how much did it grow from Q3?")
print()

response2 = agent_executor.invoke({
    "input": "Which region had the best Q4 performance and how much did it grow from Q3?",
    "chat_history": [],
    "agent_scratchpad": ""
})

print(f"\nAgent Response: {response2['output']}")

## ✍️ Hands-On Exercises

<div class="exercise-box">
<strong>🎯 Exercise 1: Build a Customer Support Agent</strong><br><br>
Create an agent that handles customer support queries by:
<ol>
<li>Looking up customer account info (account status, plan, usage)</li>
<li>Checking for known issues related to their problem</li>
<li>Generating personalized troubleshooting steps</li>
<li>Recommending upgrades if appropriate</li>
</ol>
<br>
<strong>Requirements:</strong>
<ul>
<li>Create at least 3 tools (lookup_customer, check_issues, generate_solution)</li>
<li>Create a ReAct agent with these tools</li>
<li>Test with 3 different support scenarios</li>
<li>Handle the case where the agent doesn't have enough information</li>
</ul>
</div>

In [ ]:
# Your implementation here!
# Step 1: Define customer data (simulated database)
CUSTOMER_DATA = {
    # "cust_001": {"name": "...", "plan": "...", "status": "..."},
    # Add more customers
}

# Step 2: Create tools for lookup_customer, check_issues, generate_solution
# @tool
# def lookup_customer(customer_id: str) -> str:
#     ...

# Step 3: Create agent and test
# tools = [lookup_customer, check_issues, generate_solution]
# agent = create_react_agent(llm, tools, prompt)
# agent_executor = AgentExecutor(agent=agent, tools=tools, ...)

print("Your solution here!")

<div class="exercise-box">
<strong>🎯 Exercise 2: Error Handling & Fallbacks</strong><br><br>
Enhance an agent with:
<ol>
<li>Error handling when tools fail</li>
<li>Fallback tools when primary tools don't work</li>
<li>Maximum iterations to prevent infinite loops</li>
<li>Clear error messages to the user</li>
</ol>
</div>

In [ ]:
# Your implementation here!
# def create_agent_with_error_handling(llm, tools, max_retries=3):
#     ...

print("Your solution here!")

<div class="exercise-box">
<strong>🎯 Exercise 3: Multi-Tool Agent</strong><br><br>
Build an agent with at least 5 different tools that can:
<ol>
<li>Query from different data sources</li>
<li>Combine results intelligently</li>
<li>Make decisions based on combined data</li>
</ol>
</div>

In [ ]:
# Your implementation here!
print("Your solution here!")

## 🤔 Reflection Questions

**Q1: When should you use an Agent vs a Chain?**
<details>
<summary>Click for answer</summary>
<strong>Use Chains when:</strong>
<ul>
<li>Workflow is deterministic (same steps every time)</li>
<li>You need guaranteed execution order</li>
<li>Performance/cost is critical</li>
<li>Examples: Ticket triage, data extraction</li>
</ul>
<strong>Use Agents when:</strong>
<ul>
<li>Problem requires reasoning and decision-making</li>
<li>Different inputs need different workflows</li>
<li>Agent needs to use different tools dynamically</li>
<li>Examples: Research assistant, customer support, analysis</li>
</ul>
</details>

**Q2: How do you prevent agent loops/infinite execution?**
<details>
<summary>Click for answer</summary>
<ol>
<li><strong>max_iterations:</strong> Set hard limit on iteration count</li>
<li><strong>timeout:</strong> Set time limits on tool execution</li>
<li><strong>stopping conditions:</strong> Define when agent should stop</li>
<li><strong>memory:</strong> Track what the agent has already tried</li>
<li><strong>clear tool outputs:</strong> When tools return "done", agent should stop</li>
</ol>
</details>

**Q3: How do you make agents more reliable?**
<details>
<summary>Click for answer</summary>
<ol>
<li><strong>Better prompts:</strong> Clear instructions with examples</li>
<li><strong>Better tools:</strong> Tools that return clear, unambiguous results</li>
<li><strong>Error handling:</strong> Graceful failures with recovery options</li>
<li><strong>Tool validation:</strong> Validate tool inputs before calling</li>
<li><strong>Fallbacks:</strong> Alternative tools if primary fails</li>
<li><strong>Few-shot examples:</strong> Show agent examples of good behavior</li>
</ol>
</details>

## 📝 Week 3 Project: Sales Analysis Agent

**Build a complete sales analysis agent that handles ad-hoc analyst queries.**

### Requirements:

**At least 6 Tools:**
1. Query sales by region
2. Query sales by product
3. Calculate growth rates
4. Get top performers
5. Forecast trends
6. Generate insights

**Agent Capabilities:**
- Handle complex multi-step queries
- Combine data from different sources
- Make intelligent recommendations
- Handle edge cases gracefully

**Test Queries:**
1. Simple: "What were the Q3 South region sales?"
2. Multi-step: "Which region grew the most from Q3 to Q4 and why?"
3. Predictive: "Based on trends, project Q1 sales by region"
4. Edge case: "Compare sales for invalid region X"

### Deliverables:
- 6+ well-documented tools
- ReAct agent with clear system prompt
- Test cases demonstrating agent reasoning
- Error handling and fallbacks
- Analysis of agent's reasoning process

In [ ]:
# Week 3 Project Starter

# TODO: Create 6+ tools for sales analysis
# TODO: Create ReAct agent
# TODO: Test with 4+ queries
# TODO: Implement error handling

print("🎯 Your sales analysis agent implementation here!")

## 🎓 Key Takeaways

**What you learned this week:**

✅ **ReAct Framework:**
- Think → Act → Observe → Repeat cycle
- Enables dynamic, adaptive agent behavior
- Superior to fixed chains for complex problems

✅ **Building Tools:**
- Use @tool decorator for clean tool definitions
- Clear descriptions and arguments
- Proper error handling in tool implementations

✅ **Agent Executors:**
- Handles tool selection and calling
- Manages iteration loop
- Collects agent reasoning for debugging

✅ **Real-world applications:**
- Sales analysis assistants
- Customer support automation
- Research assistants
- Data analytics chatbots

## 🔜 Next Week: RAG & Embeddings

In Week 4, we'll ground agents in your own data:
- Embeddings and vector stores
- Document retrieval (RAG)
- Building knowledge-grounded agents
- Combining agents with vector search

**Preview question:** How would you modify the sales agent to use company documents (guides, procedures) to inform its responses?

## 📚 Additional Resources

- [LangChain Agents Documentation](https://python.langchain.com/docs/modules/agents/)
- [ReAct Paper](https://arxiv.org/abs/2210.03629)
- [Tool Use Best Practices](https://python.langchain.com/docs/modules/tools/)

---

**🎉 Congratulations on completing Week 3!** You can now build autonomous agents that reason and act. See you next week! 🚀